In [3]:
import torch
from torch.autograd import Function
from torch.autograd import Variable as V
import torch as t
import numpy as np
import math
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
device = torch.device("cuda")
device1 = torch.device("cpu")
import torch.optim as optim
import time
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules import Module
from torch.nn.modules.lazy import LazyModuleMixin
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [4]:
#Mean Square Error
def MSE(pred,true):
    return np.mean((pred-true)**2)

#Root Mean Square Error
def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

#Mean Absolute Error
def MAE(pred, true):
    return np.mean(np.abs(pred-true))

#Mean Absolute Percentage Error
def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

#Relative Squared Error
def RSE(pred, true):
    return np.sqrt(np.sum((true-pred)**2)) / np.sqrt(np.sum((true-true.mean())**2))

def CORR(pred, true):
    u = ((true-true.mean(0))*(pred-pred.mean(0))).sum(0) 
    d = np.sqrt(((true-true.mean(0))**2*(pred-pred.mean(0))**2).sum(0))
    return (u/d).mean(-1)

'''
def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))
'''

#Coefficient of Determination
def R2(pred, true):
    return r2_score(pred,true) 

def gamma(a):
    
    if a <= 0 and a %1 == 0:
        return float('inf')
    
    return math.gamma(a)

def matrix_fractional_differentiation_linear(x,w,b,a):
    
    t0, t1 = t.tensor([0.]),t.tensor([1.])
    t0 = t0.to(device)
    t1 = t1.to(device)
    w_matrix_fractional_differentiation = x
    
    w_rows, w_cols = w.size()
    x_rows, x_cols = x.size()
    x = ((t.sum(x,dim=0)/x_rows).view(1,-1))
   
    y = F.linear(x,w.T,b)  
    f1 = t.mul(x,w.T).T
    c_partial = y - f1
    
    #x
    x_1 = t.where(x <= 0, t0, t1)
    x_0 = t.where(x > 0, t0, t1)
    xx = t.abs(x)
    x_partial_1 = t.mul(c_partial.T,t.mul(x_1,xx**(-a)/gamma(1-a))).T
    x_main_1 = t.mul(w.T,t.mul(x_1,xx**(1-a)/gamma(2-a))).T
    x_main_0 = t.mul(w.T,t.mul(x_0,xx**0)).T
    
    #w
    '''
    w_1 = t.where(w <= 0, t0, t1)
    w_0 = t.where(w > 0, t0, t1)
    ww = t.abs(w)
    w_partial_1 = t.mul(c_partial,t.mul(w_1,ww**(-a)/gamma(1-a)))
    w_main_1 = t.mul(x,t.mul(w_1.T,ww.T**(1-a)/gamma(2-a))).T 
    w_main_0 = t.mul(x,t.mul(w_0.T,ww.T**0)).T 
    w_matrix_fractional_differentiation = (t.sum(w_main_1+w_partial_1+w_main_0,dim=1)/w_cols).view(1,-1)
    w_matrix_fractional_differentiation = t.repeat_interleave(w_matrix_fractional_differentiation, repeats=x_rows, dim=0) 
    '''
    
    #b
    b_1 = t.where(b <= 0, t0, t1)
    b_0 = t.where(b > 0, t0, t1)
    bb = t.abs(b)
    b_c_partial = y - b
    b_partial_1 = t.mul(b_c_partial,t.mul(b_1,bb**(-a)/gamma(1-a)))
    b_main_1 = t.mul(b_1,bb**(1-a)/gamma(2-a))
    b_main_0 = t.mul(b_0,bb**0)
    
    return x_main_1+x_partial_1+x_main_0, w_matrix_fractional_differentiation, b_main_1+b_partial_1+b_main_0

def matrix_fractional_differentiation_linear_plus(x,w,b,a):
    
    w_matrix_fractional_differentiation = x
    
    w_rows, w_cols = w.size()
    x_rows, x_cols = x.size()
    x = ((t.sum(x,dim=0)/x_rows).view(1,-1))
   
    y = F.linear(x,w.T,b)  
    f1 = t.mul(x,w.T).T
    c_partial = y - f1
    
    #x
    xx = t.abs(x)
    x_partial = t.mul(c_partial.T,xx**(-a)/gamma(1-a)).T
    x_main = t.mul(w.T,xx**(1-a)/gamma(2-a)).T
    
    '''
    #w
    ww = t.abs(w)
    w_partial = t.mul(c_partial,ww**(-a)/gamma(1-a))
    w_main = t.mul(x,ww.T**(1-a)/gamma(2-a)).T 
    w_matrix_fractional_differentiation = (t.sum(w_main+w_partial,dim=1)/w_cols).view(1,-1)
    '''
    #b
    bb = t.abs(b)
    b_c_partial = y - b
    b_partial = t.mul(b_c_partial,bb**(-a)/gamma(1-a))
    b_main = bb**(1-a)/gamma(2-a)
    
    return x_main+x_partial, w_matrix_fractional_differentiation, b_main+b_partial

#This is not an important Class; its purpose is solely to obtain the weight matrix W and bias b.
#We have rearranged the positions of 'in_features' and 'out_features' for ease of future use.
class Linear(Module):

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((in_features, out_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight.T, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
#ifogd_MSELoss
class MSELoss_fractional(Function):
    
    def forward(ctx,input1,target):
        loss = nn.MSELoss()
        ctx.save_for_backward(input1,target)
        return loss(input1,target)
    
    def backward(ctx,grad_outputs):
        input1,target = ctx.saved_tensors
        t0, t1 = t.tensor([0.]),t.tensor([1.])
        t0 = t0.to(device)
        t1 = t1.to(device)
        y1 = t.where(input1 - target <= 0, t0, t1)
        y_pred_f = t.abs(input1 - target)
        grad_output1 = gamma(3)*y_pred_f**(2-a)/gamma(3-a)*y1
        y2 = t.where(input1 - target > 0, t0, t1)
        grad_output2 = 2*(input1 - target)*y2

        grad_output = (grad_output1 + grad_output2)/input1.numel()
        
        return grad_output,None
    
#ifogd_Linear+
class Linear_fractional(Function):
    
    def forward(ctx,input1,w,b):
        ctx.save_for_backward(input1,w,b)
        outputs = input1@w + b
        return outputs
    
    def backward(ctx,grad_outputs):
        input1,w,b = ctx.saved_tensors
        #w4:x_matrix_fractional_differentiation
        #x4:w_matrix_fractional_differentiation
        #b4:b_matrix_fractional_differentiation
        w4, x4, b4 = matrix_fractional_differentiation_linear(input1,w,b,a) 
        #w4, x4, b4 = matrix_fractional_differentiation_linear_plus(input1,w,b,a) 
        
        x_grad = torch.mm(grad_outputs,w4.T)
        w_grad = torch.mm(x4.T,grad_outputs)
        b_grad = torch.mul(grad_outputs,b4)
        return x_grad,w_grad,b_grad
    
    
#ifogd_Linear
class Linear_fractional_plus(Function):
    
    def forward(ctx,input1,w,b):
        ctx.save_for_backward(input1,w,b)
        outputs = input1@w + b
        return outputs
    
    def backward(ctx,grad_outputs):
        input1,w,b = ctx.saved_tensors
        #w4:x_matrix_fractional_differentiation
        #x4:w_matrix_fractional_differentiation
        #b4:b_matrix_fractional_differentiation
        #w4, x4, b4 = matrix_fractional_differentiation_linear(input1,w,b,a)   #IFOGD
        #w4, x4, b4 = matrix_fractional_differentiation_linear_plus(input1,w,b,a)   #IFOGD+
        
        x_grad = torch.mm(grad_outputs,w4.T)
        w_grad = torch.mm(x4.T,grad_outputs)
        b_grad = torch.mul(grad_outputs,b4)
        return x_grad,w_grad,b_grad
    
def split(X,y):
    X_train,X_temp,y_train,y_temp = train_test_split(X,y,test_size=0.3,shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.333,shuffle=False)
    return X_train,X_val,X_test,y_train,y_val,y_test

In [23]:
#GSPC，DJI，IXIC，FTSE，GDAXI，N225 data preprocessing
slide_windows_size = 36  #i.e.,input length
pre_days = 48     #i.e.,prediction lengths,48,96,192,384
#stock = 'GSPC'   #GSPC，DJI，IXIC，FTSE，GDAXI，N225 
stock = 'ETTm2'  #ETTh1，ETTh2，ETTm1，ETTm2
df_DJIA = pd.read_csv(r'./data/'+stock+'.csv')
#del df_DJIA['Date']
del df_DJIA['date']        #ETT
sca_DJIA = scaler.fit_transform(df_DJIA)
#label = t.tensor([row[4] for row in sca_DJIA][slide_windows_size:]).view(-1,1)  #stock
label = t.tensor([row[6] for row in sca_DJIA][slide_windows_size:]).view(-1,1)  #ETT
sca_DJIA = t.tensor(sca_DJIA[:-pre_days])
#label
deq_DJIA = deque(maxlen=pre_days)
DJIA_label = []
for i in label:
    deq_DJIA.append(list(i))
    if len(deq_DJIA)==pre_days:
        DJIA_label.append(list(deq_DJIA))
DJIA_label = t.Tensor(DJIA_label).view(-1,pre_days)
#X
deq_DJIA = deque(maxlen=slide_windows_size)
DJIA_X = []
for i in sca_DJIA:
    deq_DJIA.append(list(i))
    if len(deq_DJIA)==slide_windows_size:
        DJIA_X.append(list(deq_DJIA))
DJIA_X = t.Tensor(DJIA_X)


X_DJIA, y_DJIA = DJIA_X.to(device), DJIA_label.to(device)

X_train,X_val,X_test,y_train,y_val,y_test = split(X_DJIA,y_DJIA)   #7:2:1


class Net(nn.Module):
    def __init__(self,):
        super(Net,self).__init__()
        #self.linear1 = Linear(slide_windows_size*5,128)
        self.linear1 = Linear(slide_windows_size*7,128)  #ETT
        self.linear2 = Linear(128,64)
        self.linear3 = Linear(64,32)
        self.linear4 = Linear(32,pre_days)
        self.w1,self.b1 = self.linear1.weight,self.linear1.bias
        self.w2,self.b2 = self.linear2.weight,self.linear2.bias
        self.w3,self.b3 = self.linear3.weight,self.linear3.bias
        self.w4,self.b4 = self.linear4.weight,self.linear4.bias
    
    def forward(self,x):
        #x = x.view(-1,slide_windows_size*5)
        x = x.view(-1,slide_windows_size*7)   #ETT
        y2 = Linear_fractional.apply(x,self.w1,self.b1)   
        y3 = Linear_fractional.apply(y2,self.w2,self.b2)   
        y4 = Linear_fractional.apply(y3,self.w3,self.b3) 
        y5 = Linear_fractional.apply(y4,self.w4,self.b4)  
        return y5

In [24]:
alpha_i, alpha_f = 0, 0
net = Net()
net.load_state_dict(torch.load('./model/Adam_ETT/'+stock+'_net_fractional_1_0_'+str(pre_days)+'.pth'))  
net.to(device)
out = net(X_test)  
MSE_f_best = 10
length = len(y_test.cpu().detach().numpy())
for iii in range(0,length):
    MSE_f = MSE(y_test.cpu().detach().numpy()[iii],out.cpu().detach().numpy()[iii])
    if MSE_f<MSE_f_best:
        MSE_f_best = MSE_f
        alpha_i = iii   

net.load_state_dict(torch.load('./model/Adam_ETT/'+stock+'_net_fractional_plus_1_3_'+str(pre_days)+'.pth'))  ##
net.to(device)
out = net(X_test)  
MSE_f_best = 10
length = len(y_test.cpu().detach().numpy())
for iii in range(0,length):
    MSE_f = MSE(y_test.cpu().detach().numpy()[iii],out.cpu().detach().numpy()[iii])
    if MSE_f<MSE_f_best:
        MSE_f_best = MSE_f
        alpha_f = iii   

In [25]:
#ETTh1，ETTh2，ETTm1，ETTm2
ETTm2_true_i = (y_test.cpu().detach().numpy())[alpha_i]   ##
ETTm2_true_f = (y_test.cpu().detach().numpy())[alpha_f]   ##

net.load_state_dict(torch.load('./model/Adam_ETT/'+stock+'_net_fractional_1_0_'+str(pre_days)+'.pth'))
out = net(X_test)
ETTm2_prediction_i= (out.cpu().detach().numpy())[alpha_i]  ##

net.load_state_dict(torch.load('./model/Adam_ETT/'+stock+'_net_fractional_plus_1_3_'+str(pre_days)+'.pth'))  ##
out = net(X_test)
ETTm2_prediction_f= (out.cpu().detach().numpy())[alpha_f]  ##

In [43]:
from matplotlib.pyplot import MultipleLocator
fig = plt.figure(figsize=(10,6),dpi=1200)
custom_params = {
    'font.size': 10,
    'lines.linewidth': 1,
    'axes.labelsize': 10,
    'axes.titlesize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 8,
    'grid.linestyle': '--',
    'grid.alpha': 0.7,
}

plt.rcParams.update(custom_params)
fig.subplots_adjust(wspace=0.05, hspace=0) 

ax1 = fig.add_subplot(4, 2, 1)
ax1.plot(ETTh1_true_i,c='b')
ax1.plot(ETTh1_prediction_i,c='r')
ax1.set_xticklabels([])
ax1.set_yticklabels([])
ax1.yaxis.set_major_locator(MultipleLocator(0.1))
ax1.xaxis.set_major_locator(MultipleLocator(5))
ax1.grid()
ax1.set_ylabel(r'${ETTh1}$')
ax1.set_title(r'$(a):\alpha  = 1.0$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.4,y=0.7,fontsize='8')

ax2 = fig.add_subplot(4, 2, 2)
ax2.plot(ETTh1_true_f,c='b')
ax2.plot(ETTh1_prediction_f,c='r')
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.yaxis.set_major_locator(MultipleLocator(0.1))
ax2.xaxis.set_major_locator(MultipleLocator(5))
ax2.grid()
ax2.set_title(r'$(b):\alpha  = 1.3$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.7,y=0.7,fontsize='8')

ax3 = fig.add_subplot(4, 2, 3)
ax3.plot(ETTh2_true_i,c='b')
ax3.plot(ETTh2_prediction_i,c='r')
ax3.set_xticklabels([])
ax3.set_yticklabels([])
ax3.yaxis.set_major_locator(MultipleLocator(0.2))
ax3.xaxis.set_major_locator(MultipleLocator(5))
ax3.grid()
ax3.set_ylabel(r'${ETTh2}$')
ax3.set_title(r'$(c):\alpha  = 1.0$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.75,fontsize='8')

ax4 = fig.add_subplot(4, 2, 4)
ax4.plot(ETTh2_true_f,c='b')
ax4.plot(ETTh2_prediction_f,c='r')
ax4.set_xticklabels([])
ax4.set_yticklabels([])
ax4.yaxis.set_major_locator(MultipleLocator(0.2))
ax4.xaxis.set_major_locator(MultipleLocator(5))
ax4.grid()
ax4.set_title(r'$(d):\alpha  = 0.9$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.3,y=0.7,fontsize='8')

ax5 = fig.add_subplot(4, 2, 5)
ax5.plot(ETTm1_true_i,c='b')
ax5.plot(ETTm1_prediction_i,c='r')
ax5.set_xticklabels([])
ax5.set_yticklabels([])
ax5.yaxis.set_major_locator(MultipleLocator(0.2))
ax5.xaxis.set_major_locator(MultipleLocator(5))
ax5.grid()
ax5.set_ylabel(r'${ETTm1}$')
ax5.set_title(r'$(e):\alpha  = 1.0$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.7,fontsize='8')

ax6 = fig.add_subplot(4, 2, 6)
ax6.plot(ETTm1_true_f,c='b')
ax6.plot(ETTm1_prediction_f,c='r')
ax6.set_xticklabels([])
ax6.set_yticklabels([])
ax6.yaxis.set_major_locator(MultipleLocator(0.1))
ax6.xaxis.set_major_locator(MultipleLocator(5))
ax6.grid()
ax6.set_title(r'$(f):\alpha  = 1.1$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.8,y=0.75,fontsize='8')

ax7 = fig.add_subplot(4, 2, 7)
ax7.plot(ETTm2_true_i,c='b')
ax7.plot(ETTm2_prediction_i,c='r')
#ax7.set_xticklabels([])
ax7.set_yticklabels([])
ax7.yaxis.set_major_locator(MultipleLocator(0.02))
ax7.xaxis.set_major_locator(MultipleLocator(5))
ax7.grid()
ax7.set_ylabel(r'${ETTm2}$')
ax7.set_title(r'$(g):\alpha  = 1.0$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.4,y=0.7,fontsize='8')

ax8 = fig.add_subplot(4, 2, 8)
ax8.plot(ETTm2_true_f,c='b')
ax8.plot(ETTm2_prediction_f,c='r')
#ax8.set_xticklabels([])
ax8.set_yticklabels([])
ax8.yaxis.set_major_locator(MultipleLocator(0.05))
ax8.xaxis.set_major_locator(MultipleLocator(5))
ax8.grid()
ax8.set_title(r'$(h):\alpha  = 1.3$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.5,y=0.75,fontsize='8')

'''
ax9 = fig.add_subplot(4, 3, 9)
ax9.plot(ETTm1_true_f_p,c='b')
ax9.plot(ETTm1_prediction_f_p,c='r')
ax9.set_xticklabels([])
ax9.set_yticklabels([])
ax9.yaxis.set_major_locator(MultipleLocator(0.1))
ax9.xaxis.set_major_locator(MultipleLocator(5))
ax9.grid()
ax9.set_title(r'$(i):\alpha  = 1.1$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.45,y=0.7,fontsize='8')

ax10 = fig.add_subplot(4, 3, 10)
ax10.plot(ETTm2_true_i,c='b')
ax10.plot(ETTm2_prediction_i,c='r')
#ax10.set_xticklabels([])
ax10.set_yticklabels([])
ax10.yaxis.set_major_locator(MultipleLocator(0.01))
ax10.xaxis.set_major_locator(MultipleLocator(5))
ax10.grid()
ax10.set_xlabel(r'$IFOAdam$')
ax10.set_ylabel(r'${ETTm2}$')
ax10.set_title(r'$(j):\alpha  = 1.0$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.4,y=0.7,fontsize='8')

ax11 = fig.add_subplot(4, 3, 11)
ax11.plot(ETTm2_true_f,c='b')
ax11.plot(ETTm2_prediction_f,c='r')
ax11.set_yticklabels([])
ax11.yaxis.set_major_locator(MultipleLocator(0.05))
ax11.xaxis.set_major_locator(MultipleLocator(5))
ax11.grid()
ax11.set_xlabel(r'$IFOAdam$')
ax11.set_title(r'$(k):\alpha  = 1.3$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.4,y=0.7,fontsize='8')

ax12 = fig.add_subplot(4, 3, 12)
ax12.plot(ETTm2_true_f_p,c='b')
ax12.plot(ETTm2_prediction_f_p,c='r')
#ax12.set_xticklabels([])
ax12.set_yticklabels([])
ax12.yaxis.set_major_locator(MultipleLocator(0.05))
ax12.xaxis.set_major_locator(MultipleLocator(5))
ax12.grid()
ax12.set_xlabel(r'$IFOAdam^+$')
ax12.set_title(r'$(l):\alpha  = 1.3$',bbox=dict(facecolor='y', edgecolor='blue', alpha=0.65 ),x=0.4,y=0.7,fontsize='8')
'''

labels = ['true', 'prediction']
fig.legend(labels, loc="lower center" ,ncol=2)
plt.savefig('picture/Fig4.13.eps',bbox_inches='tight',format='eps')
plt.savefig('picture/Fig4.13.svg',bbox_inches='tight',format='svg')
plt.savefig('picture/Fig4.13.pdf',bbox_inches='tight',format='pdf')
plt.show()

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
